# LLM handbook

Following guidance from <a href='https://www.pinecone.io/learn/series/langchain/'> Pinecone's Langchain handbook.</a>

In [1]:
# import packages
import os
import langchain
import getpass
from langchain import HuggingFaceHub, LLMChain
from dotenv import load_dotenv

In [2]:
load_dotenv()

os.environ.get('HUGGINGFACEHUB_API_TOKEN');

## Skill 1 - using prompt templates

A prompt is the input to the LLM. Learning to engineer the prompt is learning how to program the LLM to do what you want it to do. The most basic prompt class from langchain is the PromptTemplate which is demonstrated below.

In [3]:
from langchain import PromptTemplate

# create template
template = """
Answer the following question: {question}

Answer:
"""

# create prompt using template
prompt = PromptTemplate(
    template=template,
    input_variables=['question']
)

The next step is to instantiate the LLM. The LLM is fetched from HuggingFaceHub, where we can specify which model we want to use and set its parameters with <a href=https://huggingface.co/docs/transformers/main_classes/text_generation>this as reference </a>. We then set up the prompt+LLM chain using langchain's LLMChain class.

In [4]:
# instantiate llm
llm = HuggingFaceHub(
    repo_id='tiiuae/falcon-7b-instruct',
    model_kwargs={
        'temperature':1.5,
        'penalty_alpha':2, 
        'top_k':5,
        'max_length': 1000
    }
)

# instantiate chain
llm_chain = LLMChain(
    llm=llm,
    prompt=prompt,
    verbose=True
)

/Users/danielsuarez-mash/anaconda3/envs/llm/lib/python3.11/site-packages/huggingface_hub/utils/_deprecation.py:127: FutureWarning: '__init__' (from 'huggingface_hub.inference_api') is deprecated and will be removed from version '0.19.0'. `InferenceApi` client is deprecated in favor of the more feature-complete `InferenceClient`. Check out this guide to learn how to convert your script to use it: https://huggingface.co/docs/huggingface_hub/guides/inference#legacy-inferenceapi-client.
  warnings.warn(warning_message, FutureWarning)


Now all that's left to do is ask a question and run the chain.

In [5]:
# define question
question = "How many champions league titles has Real Madrid won?"

# run question
print(llm_chain.run(question))



> Entering new LLMChain chain...
Prompt after formatting:

Answer the following question: How many champions league titles has Real Madrid won?

Answer:


> Finished chain.
Madrid has won the UEFA European Super League (formerly known as the SuperLiga) a record 14 times.


## Skill 2 - using chains

Chains are at the core of langchain. They represent a sequence of actions. Above, we used a simple prompt + LLM chain. Let's try some more complex chains.

### Math chain

In [6]:
from langchain.chains import LLMMathChain

llm_math_chain = LLMMathChain.from_llm(llm)

llm_math_chain.run('What is 5-3?')

'Answer: 2'

We can see what prompt the LLMMathChain class is using here. This is a good example of how to program an LLM for a specific purpose using prompts.

In [7]:
print(llm_math_chain.prompt.template)

Translate a math problem into a expression that can be executed using Python's numexpr library. Use the output of running this code to answer the question.

Question: ${{Question with math problem.}}
```text
${{single line mathematical expression that solves the problem}}
```
...numexpr.evaluate(text)...
```output
${{Output of running the code}}
```
Answer: ${{Answer}}

Begin.

Question: What is 37593 * 67?
```text
37593 * 67
```
...numexpr.evaluate("37593 * 67")...
```output
2518731
```
Answer: 2518731

Question: 37593^(1/5)
```text
37593**(1/5)
```
...numexpr.evaluate("37593**(1/5)")...
```output
8.222831614237718
```
Answer: 8.222831614237718

Question: {question}



### Transform chain

The transform chain allows transform queries before they are fed into the LLM.

In [8]:
import re 

# define function to transform query
def transform_func(inputs: dict) -> dict:

    question = inputs['question']

    question = re.sub(' +', ' ', question)

    return {'output_question': question}

In [9]:
from langchain.chains import TransformChain

# define transform chain
transform_chain = TransformChain(input_variables=['question'], output_variables=['output_question'], transform=transform_func)

# test transform chain
transform_chain.run('Hello   my name is     Daniel')

'Hello my name is Daniel'

In [10]:
from langchain.prompts import PromptTemplate
from langchain import LLMChain

In [11]:
# create new prompt to take input as 'output_question'
template = """
Answer this question: {output_question}

Answer:
"""
prompt = PromptTemplate(template=template, input_variables=['output_question'])

llm_chain.prompt = prompt

In [12]:
from langchain.chains import SequentialChain

sequential_chain = SequentialChain(chains=[transform_chain, llm_chain], input_variables=['question'])

In [15]:
print(sequential_chain.run("What     will happen     to  me if I only get 4 hours sleep tonight?"))



> Entering new LLMChain chain...
Prompt after formatting:

Answer this question: What will happen to me if I only get 4 hours sleep tonight?

Answer:


> Finished chain.
1. Reduced alertness: 4 hours is not enough sleep for adults and can result in decreased reaction times and decreased concentration during the day.
2. Poor physical and mental performance: This lack of quality rest can impair cognitive and physical abilities, leading to fatigue, poor focus, decreased reaction times and slower reflexes. This can affect both short-term and long-term physical and mental health.
3. Immunity compromised: Research has shown that lack of adequate sleep can weaken the immune system, leading to higher risk of illnesses.
4. Emotional regulation problems: Sleep deprivation can result in mood disturbances such as irritability, depression, and anxiety.
5. Increased risk for chronic health problems: Chronic lack of sleep can be linked with a variety of long-term health complications such as obesit